# Endangered Animals Image Classification



In [2]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.transforms as transforms
import torchvision.models as models
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

from PIL import Image


In [3]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
if  torch.backends.mps.is_available():
    print ("Using Apple Metal Performance Shaders")
else:
    print ("Using CPU")

torch.set_default_device(device)

Using CPU
Using: cpu


# Preparing Image  & Data


In [ ]:

transform = transforms.Compose([transforms.Resize((256,256)), transforms.ToTensor()])
imageData = datasets.ImageFolder(root='../dataset/Danger Of Extinction', transform=transform)
dataloader = DataLoader(imageData, batch_size=1)

<font size=5>Creating 2D array with image and label<font>

In [ ]:
# 2D array created to pair images with their respective label
pairArray = []
for image, label in imageData:
    pairArray.append((image, label))

<font size=5>Get size of array<font>

In [ ]:
rows = len(pairArray)
cols = len(pairArray[0])

print(f"Rows: {rows}")
print(f"Cols: {cols}")

<font size=5>View image from 2D array<font>

In [ ]:
image_view, labels = pairArray[0]

image = image_view.permute(1, 2, 0).numpy()

plt.imshow(image)
plt.title(f"Label: {labels}")
plt.axis('off')
plt.show()

<font size=5>Used to View the images from DataLoader</font>


In [ ]:
# Used to view first image
for images, label in dataloader:
    print(f"Image shape: {images.shape}")
    firstImage = images
    print(f"Label: {label}")
    firstLabel = label
    break



image_t = firstImage[0]
image_array = image_t.permute(1, 2, 0).numpy()

plt.imshow(image_array)
plt.axis('off')
plt.title(f'Label: {labels}')
plt.show()

# Data Normalization

# Model

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        # First convolutional layer & maxpooling
        self.conv1 = nn.Conv2d(3, 6, 4)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 4)
        self.fc1 = nn.Linear(16 * 4 * 4, 90)
        self.fc2 = nn.Linear(90, 61)
        self.fc3 = nn.Linear(61, 11)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # flattened tensor
        x = x.view(-1, 16 * 4 * 4)
        # call the fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc3(x))
        x = self.fc2(x)
        return x


# Training Function

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    sfsd